In [1]:
import mysql.connector
import pandas as pd
pd.set_option('display.max_columns', 10)
import numpy as np

user = "readuser2"
password = "6FbUDa5VM"
host = "otrs.futurenet.in"
port = 3306
database = 'otrs5'

db_conn = mysql.connector.connect(host='otrs.futurenet.in', port=3306, user='readuser2', password='6FbUDa5VM',
                                  database='otrs5')
cursor = db_conn.cursor()
sql_query7 = """SELECT distinct
                 t.tn as Ticket_Id,
                (select substring(cast(thi.name as char(100)),'31',position('%%OldValue%%' in thi.name)-31) as Customer from ticket_history thi where thi.ticket_id=t.id AND thi.name like '%FieldName%Customer%' and thi.create_time=(select max(thii.create_time) from ticket_history thii where thii.ticket_id=thi.ticket_id AND thii.name like '%FieldName%Customer%') limit 1) as Customer,
                t.title as Subject,
                 CASE WHEN ts.name IN ('WORK IN PROGRESS','OPEN','ON-HOLD','Waiting for Approval','Waiting for Vendor','Waiting for Customer','pending auto reopen') THEN DATEDIFF(NOW(),t.create_time) ELSE DATEDIFF(t.change_time,t.create_time) END as Age,
                 u.login as Responsible_user,
                         tt.name AS type_name

FROM

                        ticket t
                 LEFT JOIN ticket_type AS tt ON t.type_id=tt.id 
LEFT JOIN ticket_state AS ts ON t.ticket_state_id=ts.id 
LEFT JOIN queue q ON t.queue_id=q.id
LEFT JOIN users u on  t.user_id=u.id
                         WHERE
                 tt.name not in ('-')
                 AND tt.name NOT IN ('junk')
                 AND q.name not in ('PRESALES','Misc','Postmaster','ZENTINTL','TEKONCALL','CUSTOMER-ALERTS','ODOO','ODOOHELPDESK')
                  AND ts.name  in ('new','ON-HOLD','OPEN','WORK IN PROGRESS','Waiting for Approval','Waiting for Vendor','Waiting for Customer','pending auto reopen')
                    ORDER BY Age DESC"""

data = pd.read_sql(sql_query7,con=db_conn)

/tmp/ipykernel_1194516/525492776.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_query7,con=db_conn)


In [2]:
data.head()

,Ticket_Id,Customer,Subject,Age,Responsible_user,type_name
0,2023011910001249,ABAN OFFSHORE LIMITED,Confirm URL is Accessible -TCL,39,lokesh,INCIDENT
1,2023012510001826,MATRIX BUSINESS INDIA (P) LTD,Disk Usage alert-\tHYD-AD-01-Filesystem E:/,33,yashwanth,PROACTIVE NOTIFICATIONS
2,2023012810002347,HINDU MISSION HOSPITAL,Encountering an error while trying to connect ...,30,lokesh,SERVICE REQUEST
3,2023020210001411,HUMANFACTORS,Fw: Amazon Aurora MySQL 1 (with MySQL 5.6 comp...,25,lokesh,SERVICE REQUEST
4,2023020910001561,FUTURENET,App server slow - ae1,18,dinesh,INCIDENT


In [8]:
stack = data.groupby(["Responsible_user", "type_name"])["type_name"].agg(['count'])

In [9]:
stack

count
Responsible_user type_name                     
asarudeen        SERVICE REQUEST              1
balaganesh       MAINTENANCE                  1
dinesh           INCIDENT                     1
                 PROACTIVE NOTIFICATIONS      2
                 SERVICE REQUEST              1
ganeshkumar      PROACTIVE NOTIFICATIONS      2
                 SERVICE REQUEST              1
gopinath         INCIDENT                     1
karthick.m       INCIDENT                     1
                 INFORMATION                  2
                 PROACTIVE NOTIFICATIONS      1
                 SERVICE REQUEST              1
lokesh           INCIDENT                     1
                 ON-BOARDING                  1
                 REPORT                       1
                 SERVICE IMPROVEMENT          1
                 SERVICE REQUEST              2
manoj.r          INCIDENT                     1
                 PROACTIVE NOTIFICATIONS      2
naveenkumar      INCIDENT                     1
                 REPORT                       1
subeshbabu.a.k   INCIDENT                     1
yashwanth        INCIDENT                     1
                 INFORMATION                  1
                 PROACTIVE NOTIFICATIONS      4
                 SERVICE REQUEST              1

In [5]:
type(stack_total)

pandas.core.series.Series

In [6]:
stack_total = stack_total.to_frame()

In [7]:
stack_total.reset_index(inplace=True)

In [8]:
stack_total

,Responsible_user,count
0,asarudeen,1
1,balaganesh,1
2,dinesh,4
3,ganeshkumar,3
4,gopinath,1
5,karthick.m,5
6,lokesh,6
7,manoj.r,3
8,naveenkumar,2
9,subeshbabu.a.k,1


In [9]:
stack_total["Responsible_user"]

0          asarudeen
1         balaganesh
2             dinesh
3        ganeshkumar
4           gopinath
5         karthick.m
6             lokesh
7            manoj.r
8        naveenkumar
9     subeshbabu.a.k
10         yashwanth
Name: Responsible_user, dtype: object

In [10]:
stack_total["count"]


0     1
1     1
2     4
3     3
4     1
5     5
6     6
7     3
8     2
9     1
10    7
Name: count, dtype: int64

In [11]:
data["type_name"]


0                    INCIDENT
1     PROACTIVE NOTIFICATIONS
2             SERVICE REQUEST
3             SERVICE REQUEST
4                    INCIDENT
5     PROACTIVE NOTIFICATIONS
6     PROACTIVE NOTIFICATIONS
7                    INCIDENT
8                 ON-BOARDING
9                      REPORT
10    PROACTIVE NOTIFICATIONS
11                INFORMATION
12                INFORMATION
13    PROACTIVE NOTIFICATIONS
14        SERVICE IMPROVEMENT
15    PROACTIVE NOTIFICATIONS
16            SERVICE REQUEST
17            SERVICE REQUEST
18                   INCIDENT
19            SERVICE REQUEST
20            SERVICE REQUEST
21                     REPORT
22    PROACTIVE NOTIFICATIONS
23    PROACTIVE NOTIFICATIONS
24                   INCIDENT
25            SERVICE REQUEST
26                MAINTENANCE
27                   INCIDENT
28                   INCIDENT
29    PROACTIVE NOTIFICATIONS
30                INFORMATION
31    PROACTIVE NOTIFICATIONS
32                   INCIDENT
33    PROA

In [10]:
stack = data.groupby(["Responsible_user", "type_name"])["type_name"].agg(['count']).reset_index()
user_counts = stack.groupby("Responsible_user")["count"].sum().reset_index()
stack = stack.merge(user_counts, on="Responsible_user", suffixes=("", "_sum"))

In [11]:
stack

,Responsible_user,type_name,count,count_sum
0,asarudeen,SERVICE REQUEST,1,1
1,balaganesh,MAINTENANCE,1,1
2,dinesh,INCIDENT,1,4
3,dinesh,PROACTIVE NOTIFICATIONS,2,4
4,dinesh,SERVICE REQUEST,1,4
5,ganeshkumar,PROACTIVE NOTIFICATIONS,2,3
6,ganeshkumar,SERVICE REQUEST,1,3
7,gopinath,INCIDENT,1,1
8,karthick.m,INCIDENT,1,5
9,karthick.m,INFORMATION,2,5


In [12]:
stack = data.groupby(["Responsible_user", "type_name"])["type_name"].agg(['count']).reset_index()
user_counts = stack.groupby("Responsible_user")["count"].sum().reset_index()
stack = stack.merge(user_counts, on="Responsible_user", suffixes=("", "_sum"))


In [13]:
stack

,Responsible_user,type_name,count,count_sum
0,asarudeen,SERVICE REQUEST,1,1
1,balaganesh,MAINTENANCE,1,1
2,dinesh,INCIDENT,1,4
3,dinesh,PROACTIVE NOTIFICATIONS,2,4
4,dinesh,SERVICE REQUEST,1,4
5,ganeshkumar,PROACTIVE NOTIFICATIONS,2,3
6,ganeshkumar,SERVICE REQUEST,1,3
7,gopinath,INCIDENT,1,1
8,karthick.m,INCIDENT,1,5
9,karthick.m,INFORMATION,2,5


In [16]:
stack = data.groupby(["Responsible_user", "type_name"])["type_name"].agg(['count']).reset_index()
user_counts = stack.groupby("Responsible_user")["count"].sum().reset_index()
user_counts = user_counts.rename(columns={"count": "count_sum"})
stack = stack.merge(user_counts, on="Responsible_user")
stack = stack.groupby("Responsible_user").sum().reset_index()


/tmp/ipykernel_1194516/1749555823.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stack = stack.groupby("Responsible_user").sum().reset_index()


In [17]:
stack

,Responsible_user,count,count_sum
0,asarudeen,1,1
1,balaganesh,1,1
2,dinesh,4,12
3,ganeshkumar,3,6
4,gopinath,1,1
5,karthick.m,5,20
6,lokesh,6,30
7,manoj.r,3,6
8,naveenkumar,2,4
9,subeshbabu.a.k,1,1
